In [20]:
from google.colab import drive 
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [21]:
import pandas as pd
import PIL
from PIL import Image
import tensorflow as tf
from IPython.display import Image as ShowImage
from matplotlib import pyplot as plt
import os
from keras.preprocessing.image import img_to_array
from torchvision import transforms
import numpy as np

In [5]:
! unzip -q "/content/drive/MyDrive/SPOT_MASK_CHALLENGE/images.zip"

In [6]:
! ls 

drive  images  sample_data


In [22]:
train = pd.read_csv("/content/drive/MyDrive/SPOT_MASK_CHALLENGE/train_labels.csv")

In [23]:
train.head()

,image,target
0,tfymlmdkpzkqdjhdxyhnoeuqszxphw.jpg,0
1,rxgismgsvmaayzjarbfjaljhqmpbrt.jpg,1
2,uuzshfrhkgrkolhwdvliqauzulurnz.jpg,0
3,mjspxsagzusaznvnyxgamtrlqkqklp.jpg,0
4,rlbmuajgezfiddjzlyeoupxpqubkpt.jpg,1


In [24]:
train_images_list = train['image'].tolist()

In [26]:
train_images_list

['tfymlmdkpzkqdjhdxyhnoeuqszxphw.jpg',
 'rxgismgsvmaayzjarbfjaljhqmpbrt.jpg',
 'uuzshfrhkgrkolhwdvliqauzulurnz.jpg',
 'mjspxsagzusaznvnyxgamtrlqkqklp.jpg',
 'rlbmuajgezfiddjzlyeoupxpqubkpt.jpg',
 'ngvvhadzegowlxligtftkbucwyphvw.jpg',
 'qlwbzgvdyqwncrvqzaezrrqlwuugdp.jpg',
 'lbbmpcbdwgdbskcdmsanklchyknuig.jpg',
 'xgpvctqkxpuljjixphprbdrkcuzfoi.jpg',
 'tszmhququgjczggoxanwvdlzbdbxhb.jpg',
 'qvqhxwbgpskanqpsfbigdmbpkenvgk.jpg',
 'eqirckpknxclvrlxvfxaswsodbxlmm.jpg',
 'yoewqnzdgdhogcwdageakflfeogisq.jpg',
 'hpnmywojyjwgptgkflzydzenfxbtlr.png',
 'dnzimlvqpkwyhmsdwqwpuewzbdqzmg.png',
 'hcysrvramduefsgkkgvqbmxzlodgal.jpg',
 'spdcmncwcnpqagxrlivyldeknnfcvr.jpg',
 'zaemovvkdftxvorancdugwjzuodbmm.jpg',
 'uvpqnqfujwsxhbgvfbvcidhcqnyuum.jpg',
 'kzgroxmxsdfrkxusymupjaxumbwoow.jpg',
 'cjsjioqkweumqcnobaydijuvfvtrel.jpg',
 'iehmewhtlomanneoicvbyjonzzcbhh.jpg',
 'uiclughuenqkjonhhjgjclflpqkhsk.jpg',
 'hfbevhhmejjmdndclkpyeyaideyjgp.jpg',
 'pmociluihyptefnqdlihzzodiczkso.jpg',
 'qkishcozhztulvrhvodbabp

In [27]:
images_list = os.listdir('/content/images/')

In [29]:
len(images_list)

1817

In [30]:
# test_images_list = [fn for fn in images_list if fn not in train_images_list]
test_img=[]
for fn in images_list:
  if fn not in train_images_list:
    test_img.append(fn)

In [31]:
sub=pd.DataFrame({
    'image':test_img,
     'target':0
})

In [9]:
image_transform=transforms.Compose([
        transforms.Resize(size=256),
        transforms.CenterCrop(size=224)
    ])

In [10]:
train_img=[]
for i in range(0,train.shape[0]) :
    img_path="/content/images/"+str(train.iloc[i,:].image)
    image = Image.open(img_path).convert('RGB')
    image=image_transform(image)
    image = np.array(image)
    train_img.append((image))

In [11]:
target = np.array(train.target)
train_img=np.array(train_img)

In [12]:
train_img=train_img/255

In [13]:
model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(16,(3,3),input_shape=(224,224,3),activation='relu'),
        tf.keras.layers.MaxPooling2D((2,2)),
        tf.keras.layers.Conv2D(32,(3,3),activation='relu'),
        tf.keras.layers.MaxPooling2D((2,2)),
        tf.keras.layers.Conv2D(64,(3,3),activation='relu'),
        tf.keras.layers.MaxPooling2D((2,2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(units=512,activation='relu'),
        tf.keras.layers.Dense(units=1,activation='sigmoid')
    ])

from tensorflow.keras.optimizers import RMSprop

model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(lr=0.001),
              metrics=['accuracy'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [14]:
model.fit(train_img,target,epochs=2)

Epoch 1/2
41/41 [==============================] - 60s 1s/step - loss: 1.2296 - accuracy: 0.5474
Epoch 2/2
41/41 [==============================] - 58s 1s/step - loss: 0.6903 - accuracy: 0.6300


In [15]:
test_img=[]
for i in range(0,sub.shape[0]) :
    img_path="/content/images/"+str(sub.iloc[i,:].image)
    image = Image.open(img_path).convert('RGB')
    image=image_transform(image)
    image = np.array(image)
    test_img.append((image))

In [16]:
test_img=np.array(test_img)/255

In [17]:
predictions=model.predict(test_img)

In [18]:
sub.target=predictions

In [19]:
sub.to_csv('image.csv',index=False)